In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import npc_lims

In [ ]:
session_table_path=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\CO decoding results\session_table_v0.265.csv"
# session_table_path="/Users/ethan.mcbride/Data/DR/session_table_v0.265.csv"
session_table=pl.read_csv(session_table_path)

dr_session_list=(
    session_table.filter(
    pl.col('project')=="DynamicRouting",
    pl.col('is_production'),
    pl.col('is_annotated'),
    pl.col('issues')=="",
    # pl.col('is_good_behavior').eq(True),
    )['session_id'].to_list()
)

In [ ]:
templeton_session_list=(
    session_table.filter(
        pl.col('project')=="Templeton",
        pl.col('is_production'),
        pl.col('is_annotated'),
        pl.col('issues')=="",
        # pl.col('is_good_behavior').eq(True),
    )['session_id'].to_list()
)
templeton_session_list

In [ ]:
all_trials=pd.read_parquet(
    npc_lims.get_cache_path('trials')
)


In [ ]:
all_trials.query(
    'session_id in @templeton_session_list'
)

In [ ]:
sel_session_id=templeton_session_list[10]
# examples with disengaged behavior: 10, 13, 14, 17
sel_session_trials=all_trials.query(
    'session_id == @sel_session_id'
)

fig,ax=plt.subplots(1,1,figsize=(10,5))

ax.vlines(
    sel_session_trials.query('is_response')['response_time'],
    ymin=0,
    ymax=1,
    color='black',
    alpha=0.5,
    label='response'
)

ax.vlines(
    sel_session_trials.query('is_target')['stim_start_time'],
    ymin=1,
    ymax=2,
    color='red',
    alpha=0.5,
    label='target'
)

ax.legend()

ax.set_title(f"Session {sel_session_id}")

In [ ]:
rewarded_stimuli=sel_session_trials.query('is_rewarded')['stim_name'].unique()

if len(rewarded_stimuli) == 1:
    rewarded_stim_name = rewarded_stimuli[0]
    print(f"Session {sel_session_id} rewarded stimulus: {rewarded_stim_name}")
else:
    rewarded_stim_name = np.nan
    print(f"Session {sel_session_id} has multiple or no rewarded stimuli: {rewarded_stimuli}")

#get responses per minute
n_resp_thresh=4
binsize=120 # seconds
time_bins=np.arange(
    sel_session_trials['stim_start_time'].min(),
    sel_session_trials['stim_start_time'].max()+ binsize,
    binsize
)

fig,ax=plt.subplots(1,1,figsize=(10,5))
ax.axhline(n_resp_thresh, color='red', linestyle='--', label='response threshold')
ax.hist(
    sel_session_trials.query('is_response and stim_name==@rewarded_stim_name')['response_time'],
    bins=time_bins,
    color='blue',
    alpha=0.5,
    label='responses to rewarded stimulus'
)

ax.hist(
    sel_session_trials.query('stim_name==@rewarded_stim_name')['stim_start_time'],
    bins=time_bins,
    color='gray',
    alpha=0.5,
    label='all occurrences of rewarded stimulus'
)

ax.set_xlabel('Time (s)')
ax.set_ylabel('Count')
ax.set_title(f"Responses per {binsize} seconds for session {sel_session_id} with stimulus {rewarded_stim_name}")
ax.legend()


In [ ]:

time_window=120  # seconds before and after the trial

fraction_responded_trials=[]

#loop through trials
for tr, trial in sel_session_trials.iterrows():
    #before and after
    time_window_trials=sel_session_trials.query(
        f'stim_start_time >= {trial["stim_start_time"] - time_window} and '
        f'stim_start_time < {trial["stim_start_time"] + time_window}'
    )
    # # only before
    # time_window_trials = sel_session_trials.query(
    #     f'stim_start_time >= {trial["stim_start_time"] - time_window} and '
    #     f'stim_start_time < {trial["stim_start_time"]}'
    # )

    #find the number of rewarded stimuli in the time window before and after each trial
    n_reward_stimuli = time_window_trials.query('stim_name == @rewarded_stim_name').shape[0]

    #find the number of responses to rewarded stimuli in the time window before and after each trial
    n_responses = time_window_trials.query('is_response and stim_name == @rewarded_stim_name').shape[0]

    #compute the fraction
    if n_reward_stimuli > 0:
        fraction_responded = n_responses / n_reward_stimuli
    else:
        fraction_responded = np.nan  # or some other value to indicate no stimuli

    fraction_responded_trials.append({
        'trial_index': tr,
        'stim_start_time': trial['stim_start_time'],
        'n_total_trials': time_window_trials.shape[0],
        'n_reward_stimuli': n_reward_stimuli,
        'n_responses': n_responses,
        'fraction_responded': fraction_responded
    })

fraction_responded_df = pd.DataFrame(fraction_responded_trials)
# Display the DataFrame
fraction_responded_df

In [ ]:
frac_responded_thresh=0.8

fig,ax=plt.subplots(2,1,figsize=(10,5), sharex=True)

ax[0].vlines(
    sel_session_trials.query('is_response')['response_time'],
    ymin=0,
    ymax=1,
    color='tab:blue',
    alpha=0.5,
    label='response'
)

ax[0].plot(
    fraction_responded_df['stim_start_time'],
    fraction_responded_df['fraction_responded'],
    color='k',
    linewidth=2,
)

ax[0].axhline(frac_responded_thresh, color='r', linestyle='--', label='fraction responded threshold')

ax[0].set_ylabel('fraction responded')

ax[1].vlines(
    sel_session_trials.query('is_response')['response_time'],
    ymin=0,
    ymax=1,
    color='tab:blue',
    alpha=0.5,
    label='response'
)

ax[1].plot(
    fraction_responded_df['stim_start_time'],
    fraction_responded_df['fraction_responded']>= frac_responded_thresh,
    color='r',
    linewidth=2,
)

ax[1].set_ylabel('pass engagement threshold')